In [6]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time
import datetime

In [7]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [8]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]

def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

def pad(data):
  length = 16 - (len(data) % 16)
  return data + chr(length).encode('utf-8')*length

def encodeMessage(msg):
  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Erster Schritt: AES256 verschlüsseln
  iv = Random.new().read(AES.block_size)
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=iv)
  msg = msg.encode('utf-8')
  crypt = iv + cipher.encrypt(pad(msg))

  # Zweiter Schritt: base64 enkodieren
  msgb64 = base64.b64encode(crypt).decode('utf-8')
  return msgb64

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln
    msg = decodeMessage(contents.read())
  
    #JSON-String umwandeln in dict
    folder_dict = json.loads(msg)
    
    # Verbindung trennen
    connection.close()

    return folder_dict

#Funktion zur erstellung des SQL-Statements
def sql_data(contents, cur):
  #content wird Zeile für Zeile verarbeitet
  for line in contents.read().split(b'\n'):
            if line != b'':
                #Message wird dekodiert
                line = decodeMessage(line)
                print(line)

                #Zeile wird in ein dict konvertiert
                line= json.loads(line)

                columns = ""
                values = ""

                for type in line:
                  #Diese Daten werden herausgefiltert und nicht mit in die Datenbank mit aufgenommen
                  if type == "bat" or type == "DeviceID" or type == "uptime" or type == "window_event" or type == "door_event" or type == "person" or type == "window_open" or type == "Status":
                    continue
                   
                  #Erstellt den String für die Columns
                  columns += type 

                  #Erstellt den String für die Values 
                  
                  #Edge-case: wenn value eine Liste ist
                  if isinstance(line[type], list):
                    values += str(line[type][0])
                  elif type == "timestamp":
                     values += "FROM_UNIXTIME(%s)" % (int(line[type]/1000))
                  else:  
                    values += str(line[type])

                  #Edge-case: Am Ende kein Komma
                  if type != "cnt0_3":  
                    columns += ", "
                    values += ", "

                 

                sql = "INSERT INTO %s ( %s ) VALUES ( %s );" % ('measurements', columns, values)
                print(sql)
                cur.execute(sql)
            break #nachher löschen
  #Status noch aus Schema löschen

In [9]:
def file_to_db():
    folder = getAvailable()

    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)
    
    """
    for year in folder:
        print(folder[year])

    """
    for year in folder.keys():
        for month in folder[year].keys():
            for day in folder[year][month].keys():
                for file in folder[year][month][day]:
                    # Anfrage formulieren und Daten anfordern
                    print("_____________________________________________________________________________")
                    print("Year: " + year + "\n" + "Month: " + month + "\n" + "Day: " + day + "\n" + "File: " + file + "\n")   
                    connection.request("GET","/file?request="+encodeMessage(year+"/"+month+"/"+day+"/"+file))
                    contents = connection.getresponse()
                    if contents.status == 200:
                        print("Status: " + str(contents.status) + "/OK")
                    else:
                        print("Status: " + str(contents.status) + "/Canceled!")
                        break
                    sql_data(contents, cur)


    
    """
    # Daten zeilenweise entschlüsseln und in Datei "1589256960.txt" ausgeben
    with open("1679878517.txt","w") as f:
        for line in contents.read().split(b'\n'):
            if line != b'':
                f.write(decodeMessage(line)+'\n')
    """



    # Verbindung trennen
    connection.close()

In [10]:
file_to_db()

_____________________________________________________________________________
Year: 2023
Month: 3
Day: 25
File: 1679702382

Status: 200/OK
{"TypPS": 15.0, "oxygen": [20.887, 2.44], "bat": [0, 0], "pm10": [0.0, 10.0], "cnt0_5": [2.769, 10.25], "co": [1.49, 0.19], "temperature": [19.481, 0.54], "performance": 964, "uptime": 202587, "window_event": 0, "co2": [494.367, 64.83], "measuretime": 1769, "DeviceID": "de7fd19cce1c108a714ab9063e3a7518", "so2": [-21.036, 88.71], "no2": [47.203, 3.85], "cnt5": [0.0, 10.0], "timestamp": 1679702382000, "pm1": [0.0, 10.0], "door_event": 0, "cnt1": [0.0, 10.0], "dewpt": [8.239, 1.0], "Status": "OK", "tvoc": [162.269, 24.34], "pressure": [980.556, 1.0], "cnt10": [0.0, 10.0], "dCO2dt": -4.3, "sound_max": 51.3, "health": 819, "temperature_o2": [23.464, 1.0], "cnt2_5": [0.0, 10.0], "o3": [10.928, 1.27], "humidity": [49.251, 3.74], "dHdt": -0.01, "person": 0, "window_open": 0, "humidity_abs": [8.289, 0.52], "sound": [48.288, 3.46], "pm2_5": [0.0, 10.0], "cnt0

KeyboardInterrupt: 

In [ ]:
#DB-Connection close
conn.close()